In [125]:
import requests
import pandas as pd
import re
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
import numpy as np
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'fr-FR,fr;q=0.9,en;q=0.8',
    'Referer': 'https://www.allocine.fr/'
}

In [746]:
# Colonnes attendues
columns = ['Rang', 'NbSemaine', 'Titre', 'Entrées', 'Variation', 'Cumul', 'Budget', 'NbCopies', 'MoyCopies']

# Créer la liste de dictionnaires directement
data = []

for i in range(52) :
    if i < 9 :
        url = "http://www.cine-directors.net/box/2025/boxoff0"+str(i+1)+".html"
    else :
        url = "http://www.cine-directors.net/box/2025/boxoff"+str(i+1)+".html"
    page = requests.get(url, headers=headers)
    soup = bs(page.content, 'lxml')
    print(url)
    # Récupérer directement le tbody
    tbody = soup.find('tbody').find_next('tbody')
    rows = tbody.find_all('tr')[4:]  # Ignorer les 3 premières lignes


    for row in rows:
        cells = row.find_all('td')

        # Vérifier qu'on a bien 9 colonnes
        if len(cells) == 9:
            row_data = {col: cells[i].get_text().strip() for i, col in enumerate(columns)}
            
            text = tbody.find('table').find('td').get_text().strip()
            cleaned = ' '.join(text.split())
            row_data['Date'] = re.search(r'du\s+(.+?)\)', cleaned).group(1)
            
            data.append(row_data)

    
    # Créer le DataFrame une seule fois (BEAUCOUP plus rapide)
BoxOffice = pd.DataFrame(data)


http://www.cine-directors.net/box/2025/boxoff01.html
http://www.cine-directors.net/box/2025/boxoff02.html
http://www.cine-directors.net/box/2025/boxoff03.html
http://www.cine-directors.net/box/2025/boxoff04.html
http://www.cine-directors.net/box/2025/boxoff05.html
http://www.cine-directors.net/box/2025/boxoff06.html
http://www.cine-directors.net/box/2025/boxoff07.html
http://www.cine-directors.net/box/2025/boxoff08.html
http://www.cine-directors.net/box/2025/boxoff09.html
http://www.cine-directors.net/box/2025/boxoff10.html
http://www.cine-directors.net/box/2025/boxoff11.html
http://www.cine-directors.net/box/2025/boxoff12.html
http://www.cine-directors.net/box/2025/boxoff13.html
http://www.cine-directors.net/box/2025/boxoff14.html
http://www.cine-directors.net/box/2025/boxoff15.html
http://www.cine-directors.net/box/2025/boxoff16.html
http://www.cine-directors.net/box/2025/boxoff17.html
http://www.cine-directors.net/box/2025/boxoff18.html
http://www.cine-directors.net/box/2025/boxoff1

In [747]:
BoxOffice = BoxOffice[BoxOffice['Rang']!='']
BoxOffice = BoxOffice.replace(r' +\(FT\)', '', regex=True)

#On nettoie la colonne NbCopies
BoxOffice['NbCopies'] = BoxOffice['NbCopies'].str.replace(' ', '')
BoxOffice['NbCopies']= BoxOffice['NbCopies'].replace(['?','-',''], np.nan)
BoxOffice['NbCopies']= BoxOffice['NbCopies'].astype('Int64')

#On nettoie la colonne MoyCopies
BoxOffice['MoyCopies'] = BoxOffice['MoyCopies'].str.replace(' ', '')
BoxOffice['MoyCopies'] = BoxOffice['MoyCopies'].replace(['?','-',''], np.nan)
BoxOffice['MoyCopies'] = BoxOffice['MoyCopies'].astype('Int64')

#On nettoie la colonne Entrées
BoxOffice['Entrées'] = BoxOffice['Entrées'].str.replace(' ', '')
BoxOffice['Entrées'] = BoxOffice['Entrées'].replace('77entrées', '77')
BoxOffice['Entrées'] = BoxOffice['Entrées'].replace(['?','-',''], np.nan)
BoxOffice['Entrées'] = BoxOffice['Entrées'].astype('Int64')

#On nettoie la colonne Cumul
BoxOffice['Cumul'] = BoxOffice['Cumul'].str.replace(',','')
BoxOffice['Cumul'] = BoxOffice['Cumul'].astype('Int64')*1000
BoxOffice.loc[(BoxOffice['Titre'] == 'Demon slayer : La forteresse infinie - Film 1') & (BoxOffice['Rang'] == '1'),'Cumul']/=10 

#On nettoie la colonne Budget
mask2 = ~BoxOffice['Budget'].str.contains('M\$', regex=True, na=False)
BoxOffice.loc[mask2, 'Budget'] = BoxOffice.loc[mask2, 'Budget'].str.replace(' M\€', '', regex=True).str.replace(',','.').replace('-',np.nan).replace('- M£',np.nan).replace('',np.nan)

mask = BoxOffice['Budget'].str.contains('M\$', regex=True, na=False)
BoxOffice.loc[mask, 'Budget'] = (BoxOffice.loc[mask, 'Budget'].str.replace(' M\$', '', regex=True).replace('-',np.nan).str.replace(',','.'))

BoxOffice['Budget'] = BoxOffice['Budget'].astype(float)
BoxOffice['Budget'] = BoxOffice['Budget']*1000000
BoxOffice.loc[mask, 'Budget'] = BoxOffice.loc[mask, 'Budget'] * 1.20

BoxOffice['NbSemaine'] = BoxOffice['NbSemaine'].astype(int)

BoxOffice['Ratio'] = BoxOffice['Cumul']/BoxOffice['Budget']

In [791]:
BoxOffice.head()

,Rang,NbSemaine,Titre,Entrées,Variation,Cumul,Budget,NbCopies,MoyCopies,Date,Ratio
0,1,3,Mufasa : Le roi lion,986000,- 28 %,3574000,240000000.0,951,1037,1er au 7 janvier 2025,0.014892
1,2,2,"Sonic 3, le film",655000,- 47 %,1879000,146400000.0,772,848,1er au 7 janvier 2025,0.012835
2,3,6,Vaiana 2,557000,- 43 %,7506000,180000000.0,1063,524,1er au 7 janvier 2025,0.0417
3,4,1,Un ours dans le Jura,453000,New,453000,12100000.0,580,782,1er au 7 janvier 2025,0.037438
4,5,6,En fanfare,244000,- 19 %,1854000,6100000.0,642,379,1er au 7 janvier 2025,0.303934


In [748]:
BoxOffice.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1368 entries, 0 to 1418
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Rang       1368 non-null   object 
 1   NbSemaine  1368 non-null   int64  
 2   Titre      1368 non-null   object 
 3   Entrées    1367 non-null   Int64  
 4   Variation  1368 non-null   object 
 5   Cumul      1368 non-null   Int64  
 6   Budget     1070 non-null   float64
 7   NbCopies   1361 non-null   Int64  
 8   MoyCopies  1361 non-null   Int64  
 9   Date       1368 non-null   object 
dtypes: Int64(4), float64(1), int64(1), object(4)
memory usage: 122.9+ KB


In [720]:
#Quels films ont fait le plus d'entrées en une semaine ?
BoxOffice[BoxOffice['Entrées'] != np.nan][['Titre','Entrées','NbSemaine','Date']].sort_values('Entrées', ascending=False).head(10)

,Titre,Entrées,NbSemaine,Date
1367,Avatar : De feu et de cendres,2541000,1,17 au 23 décembre 2025
1393,Avatar : De feu et de cendres,2516000,2,24 au 30 décembre 2025
1283,Zootopie 2,1736000,1,26 novembre au 2 décembre 2025
579,Lilo & Stitch,1349000,2,28 mai au 3 juin 2025
553,Lilo & Stitch,1325000,1,21 au 27 mai 2025
1394,La femme de ménage,1312000,1,24 au 30 décembre 2025
1368,Zootopie 2,1284000,4,17 au 23 décembre 2025
1309,Zootopie 2,1269000,2,3 au 9 décembre 2025
1395,Zootopie 2,1269000,5,24 au 30 décembre 2025
977,The conjuring : l'heure du jugement,1109000,1,10 au 16 septembre 2025


In [732]:
#Quels films ont fait le plus d'entrée en 2025 ?
BoxOffice[BoxOffice['Cumul'] != np.nan][['Titre','Cumul','NbSemaine','Date']].sort_values('Cumul', ascending=False).drop_duplicates(subset='Titre').head(10)
#D'autres films comme Avatar : La voie de l'eau ou Star wars sont dans la liste, car ils sont ressortis en 2025, mais ils ont quand même gardé leur nombre d'entrées

,Titre,Cumul,NbSemaine,Date
1116,Avatar : La voie de l'eau,14510000,1,8 au 14 octobre 2025
239,Vaiana 2,8066000,14,25 février au 4 mars 2025
1395,Zootopie 2,6547000,5,24 au 30 décembre 2025
322,Mufasa : Le roi lion,5233000,14,19 au 25 mars 2025
920,Lilo & Stitch,5137000,14,20 au 26 août 2025
1393,Avatar : De feu et de cendres,5057000,2,24 au 30 décembre 2025
47,L'amour ouf,4901000,13,8 au 14 janvier 2025
446,Star wars : Episode 3 - La revanche des Sith,3452000,1,23 au 29 avril 2025
1059,F1,3290000,14,24 au 30 septembre 2025
46,Gladiator 2,3036000,9,8 au 14 janvier 2025


In [750]:
#Quels films sont restés le plus longtemps au cinéma ?
BoxOffice[['Titre','NbSemaine']].sort_values('NbSemaine', ascending=False).drop_duplicates('Titre').head(10)

,Titre,NbSemaine
550,Conclave,25
131,Emilia Perez,24
134,Les graines du figuier sauvage,20
292,Anora,20
133,L'histoire de Souleymane,17
264,En fanfare,15
1059,F1,14
920,Lilo & Stitch,14
322,Mufasa : Le roi lion,14
239,Vaiana 2,14


In [716]:
#Quels films ont le plus grand nombre de copies ?
BoxOffice[BoxOffice['NbCopies'] != np.nan][['Titre','NbCopies','Date']].sort_values('NbCopies', ascending=False).head(10)

,Titre,NbCopies,Date
1395,Zootopie 2,1377,24 au 30 décembre 2025
1368,Zootopie 2,1275,17 au 23 décembre 2025
717,Dragons,1126,2 au 8 juillet 2025 - Fête du cinéma bis
746,Elio,1104,9 au 15 juillet 2025
744,Dragons,1084,9 au 15 juillet 2025
662,Lilo & Stitch,1083,18 au 24 juin 2025
443,"Minecraft, le film",1081,23 au 29 avril 2025
898,Les bad guys 2,1071,20 au 26 août 2025
2,Vaiana 2,1063,1er au 7 janvier 2025
774,Elio,1060,16 au 22 juillet 2025


In [717]:
#Quels films ont le plus moins  de copies ?
BoxOffice[BoxOffice['NbCopies'] != np.nan][['Titre','NbCopies','Date']].sort_values('NbCopies', ascending=False).tail(10)

,Titre,NbCopies,Date
133,L'histoire de Souleymane,67,29 janvier au 4 février 2025
497,Ne zha 2,61,30 avril au 6 mai 2025
652,Le rendez-vous de l'été,56,11 au 17 juin 2025
43,Personne n'y comprend rien,52,8 au 14 janvier 2025
626,Zion,49,4 au 10 juin 2025
624,Jardin d'été,26,4 au 10 juin 2025
686,Thunderbolts*,26,18 au 24 juin 2025
684,Sinners,26,18 au 24 juin 2025
134,Les graines du figuier sauvage,25,29 janvier au 4 février 2025
685,Until dawn,24,18 au 24 juin 2025


In [749]:
#Quels films ont le plus grand nombre de copies ?
BoxOffice[BoxOffice['MoyCopies'] != np.nan][['Titre','MoyCopies','Date']].sort_values('MoyCopies', ascending=False).head(10)

,Titre,MoyCopies,Date
1367,Avatar : De feu et de cendres,3490,17 au 23 décembre 2025
1393,Avatar : De feu et de cendres,3118,24 au 30 décembre 2025
1283,Zootopie 2,2691,26 novembre au 2 décembre 2025
977,The conjuring : l'heure du jugement,2526,10 au 16 septembre 2025
553,Lilo & Stitch,2246,21 au 27 mai 2025
579,Lilo & Stitch,2212,28 mai au 3 juin 2025
1394,La femme de ménage,2140,24 au 30 décembre 2025
1309,Zootopie 2,1888,3 au 9 décembre 2025
389,Zion,1833,9 au 15 avril 2025
689,F1,1735,25 juin au 1er juillet 2025


In [759]:
#Quels films a la meilleur ratio Entrées/Budget ?
BoxOffice.sort_values('Ratio', ascending = False).drop_duplicates(subset='Titre').head(10)

,Rang,NbSemaine,Titre,Entrées,Variation,Cumul,Budget,NbCopies,MoyCopies,Date,Ratio
378,-,6,A bicyclette !,17000,- 56 %,525000,520000.0,253,67,2 au 8 avril 2025,1.009615
1280,-,8,Sacré-coeur,14000,- 55 %,455000,690000.0,238,58,19 au 25 novembre 2025,0.65942
1194,-,5,Un simple accident,52000,- 4 %,620000,1000000.0,440,119,29 octobre au 4 novembre 2025,0.62
264,-,15,En fanfare,19000,- 48 %,2583000,6100000.0,287,65,5 au 11 mars 2025,0.423443
1398,6,2,Le chant des forêts,199000,- 5 %,409000,1200000.0,369,539,24 au 30 décembre 2025,0.340833
160,-,9,Vingt dieux,19000,- 28 %,897000,2800000.0,224,83,5 au 11 février 2025,0.320357
1025,-,5,La nuit des clowns,16000,- 37 %,306000,1200000.0,166,99,17 au 23 septembre 2025,0.255
1170,-,7,Sirat,25000,- 32 %,673000,3000000.0,238,105,22 au 28 octobre 2025,0.224333
133,-,17,L'histoire de Souleymane,3000,- 92 %,582000,2600000.0,67,46,29 janvier au 4 février 2025,0.223846
1363,-,7,La femme le plus riche du monde,10000,- 55 %,900000,4200000.0,218,46,10 au 16 décembre 2025,0.214286


In [765]:
#Quels films ont les pire ratio Entrées/Budget ?
BoxOffice.sort_values('Ratio').drop_duplicates(subset='Titre').head(10)

,Rang,NbSemaine,Titre,Entrées,Variation,Cumul,Budget,NbCopies,MoyCopies,Date,Ratio
456,14,1,Ne zha 2,46000,New,46000,96000000.0,222,206,23 au 29 avril 2025,0.000479
258,15,1,In the lost lands,32000,New,32000,66000000.0,247,131,5 au 11 mars 2025,0.000485
1079,16,1,A big bold beautiful journey,35000,New,35000,54000000.0,207,167,1er au 7 octobre 2025,0.000648
1094,3,1,Tron : Ares,200000,New,200000,216000000.0,487,411,8 au 14 octobre 2025,0.000926
664,4,1,Elio,168000,New,168000,180000000.0,592,283,18 au 24 juin 2025,0.000933
1191,18,1,Smashing machine,57000,New,57000,60000000.0,336,170,29 octobre au 4 novembre 2025,0.00095
82,4,1,Better man,142000,New,142000,132000000.0,530,268,22 au 28 janvier 2025,0.001076
498,-,1,Star wars : Episode 3 - La revanche des Sith,0,New,149000,135600000.0,465,321,30 avril au 6 mai 2025,0.001099
1161,16,1,Springsteen : Deliver me from nowhere,76000,New,76000,66000000.0,292,259,22 au 28 octobre 2025,0.001152
108,-,1,Toutes pour une,12000,New,12000,9900000.0,155,75,22 au 28 janvier 2025,0.001212


In [789]:
#Quels films sont restés le plus longtemps en top 1 du box office ?

BoxOfficeRang = BoxOffice[BoxOffice['Rang']=='1'].copy()
BoxOfficeRang['Rang'] = BoxOfficeRang['Rang'].astype(int)

result = BoxOfficeRang.groupby('Titre')['Rang'].agg('sum').sort_values(ascending = False)
result[result == result.max()]

Titre
God save the Tuche      4
Minecraft, le film      4
Mufasa : Le roi lion    4
Les bad guys 2          4
Name: Rang, dtype: int64

In [773]:
BoxOfficeVar = BoxOffice[BoxOffice['Variation'] != 'New']

BoxOfficeVar['Variation'] = BoxOfficeVar['Variation'].str.replace('%','').str.replace(' ','')
BoxOfficeVar = BoxOfficeVar[~BoxOfficeVar['Variation'].str.contains("++", regex=False)]
BoxOfficeVar['Variation'] = BoxOfficeVar['Variation'].replace('+','')
BoxOfficeVar['Variation']=pd.to_numeric(BoxOfficeVar['Variation'], errors='coerce')
BoxOfficeVar = BoxOfficeVar[BoxOfficeVar['NbSemaine'] == 2]

In [548]:
variation[variation['Titre']=="Ollie"]

,Rang,NbSemaine,Titre,Entrées,Variation,Cumul,Budget,NbCopies,MoyCopies,Date
604,-,2,Ollie,1000,-86.0,11000.0,- M€,348,4,28 mai au 3 juin 2025


In [322]:
test['Entrées'] = test['Entrées'].replace('77 entrées','')

In [554]:
test[test['Titre']=="Ollie"]

,Rang,NbSemaine,Titre,Entrées,Variation,Cumul,Budget,NbCopies,MoyCopies,Date
571,19,1,Ollie,10000,New,10000.0,- M€,122,82,21 au 27 mai 2025
604,-,2,Ollie,1000,- 86 %,11000.0,- M€,348,4,28 mai au 3 juin 2025


In [584]:
test['Budget'] = test['Budget'].str.replace(' M€','')
test['Budget'] = test['Budget'].str.replace(' M$','')
test['Budget'] = test['Budget'].str.replace(',','.')
test['Budget'] = test['Budget'].replace('-',np.nan)
test['Budget'] = test['Budget'].replace('- M£',np.nan)
test['Budget'] = test['Budget'].astype(float)